In [4]:
import numpy as np
import pandas as pd
import scipy as sp
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.offline import init_notebook_mode

In [3]:
from pymarkowitz import *

In [5]:
init_notebook_mode(connected=True)

In [6]:
%load_ext autoreload
%autoreload 2

In [7]:
sp500 = pd.read_csv("datasets/sp500_1990_2000.csv", index_col='DATE').drop(["Unnamed: 0"], axis=1)
sp500.head()

,AAPL,ABMD,ABT,ADBE,ADM,ADSK,AEP,AFL,AIG,AJG,...,WEC,WFC,WHR,WMB,WMT,WY,XOM,XRAY,XRX,ZION
DATE,,,,,,,,,,,,,,,,,,,,,
01/02/1990,1.330,5.875,3.874,1.282,8.4918,5.0313,33.125,1.1750,148.9705,6.1875,...,10.6667,2.8281,32.250,4.9662,5.9219,11.6129,12.5000,1.0417,25.4128,3.5000
01/03/1990,1.339,5.813,3.888,1.345,8.5382,5.1563,33.000,1.1500,148.7940,6.1563,...,10.4583,2.7031,32.375,5.0755,5.8906,11.3571,12.3750,1.0417,25.5775,3.5625
01/04/1990,1.344,5.625,3.881,1.408,8.4918,5.0313,32.500,1.1250,146.6759,6.1563,...,10.4167,2.6719,32.750,4.9194,5.8594,11.2548,12.2500,1.0000,25.6324,3.5625
01/05/1990,1.348,5.750,3.839,1.440,8.0741,5.0313,31.750,1.1250,142.2633,6.1563,...,10.2500,2.5938,32.000,4.9350,5.7969,11.2036,12.2188,1.0625,25.6873,3.5625
01/08/1990,1.357,5.750,3.839,1.456,8.2134,4.9844,31.625,1.1333,141.3807,6.1563,...,10.1667,2.6719,32.250,4.9974,5.8750,11.2036,12.3750,1.0208,25.5226,3.5625


## Optimization
- In this demo. I will go through how to use the tools to construct optimization problems with constraints and objectives


### Initialization
- Select 15 random stocks with 1000 observations
- Setting up historical mean return, comoment matrix and beta with ReturnGenerator and MomentGenerator
- Examine their price movements & Return

In [7]:
selected = sp500.iloc[:1000, np.random.choice(np.arange(0, sp500.shape[1]), 15, replace=False)]
selected.head()

,BMY,APA,DTE,AON,HUM,ROL,JNJ,LOW,FISV,PVH,AJG,VAR,SNA,IBM,DOV
DATE,,,,,,,,,,,,,,,
01/02/1990,13.5628,7.5216,25.625,12.556,10.420,1.0681,7.4844,0.9141,0.4719,7.9375,6.1875,1.1053,22.000,24.5313,6.0890
01/03/1990,13.8305,7.6299,25.375,12.519,10.391,1.0681,7.5156,0.9102,0.4829,7.9375,6.1563,1.1179,21.833,24.7188,5.9844
01/04/1990,13.7413,7.1970,25.000,12.333,10.302,1.0608,7.5313,0.9141,0.4938,8.1250,6.1563,1.1179,21.750,25.0000,6.0053
01/05/1990,13.3546,6.9805,24.875,12.074,10.272,1.0608,7.4531,0.9180,0.4993,8.0000,6.1563,1.0989,22.000,24.9375,5.9844
01/08/1990,13.4141,7.4134,24.750,11.852,10.183,1.0535,7.5781,0.9297,0.4993,8.1250,6.1563,1.1116,21.917,25.1250,6.0262


In [19]:
px.line(pd.melt(selected.reset_index(), id_vars='DATE'), x='DATE', y='value', color='variable', title='Price from 1990 - 1993')

In [21]:
px.line(pd.melt((selected/selected.iloc[0]).reset_index(), id_vars='DATE'), x='DATE', y='value', color='variable', title='Cumulative from 1990 - 1993')

In [8]:
ret_generator = ReturnGenerator(selected)

In [9]:
mu_return = ret_generator.calc_mean_return(method='geometric')

In [23]:
daily_return = ret_generator.calc_return(method='daily')

In [24]:
mom_generator = MomentGenerator(daily_return)

In [25]:
cov_matrix = mom_generator.calc_cov_mat(method='exp', decay=0.94, span=30)

In [26]:
beta_vec = mom_generator.calc_beta(beta_vec=sp500.iloc[:1000].pct_change().dropna(how='any').sum(axis=1)/sp500.shape[1], method='exp', decay=0.94, span=30)

In [48]:
px.bar(pd.DataFrame(mu_return, columns=['MEAN_RETURN']).reset_index(), x='index', y='MEAN_RETURN', title='Mean Return of Selected Stocks')

In [47]:
px.bar(pd.DataFrame(beta_vec, columns=['BETA']).reset_index(), x='index', y='BETA', title='Beta of selected stocks')

In [30]:
cov_matrix # Annualized

,PCAR,GIS,BKR,HRB,CMI,CINF,LLY,UTX,CMCSA,LB,MXIM,USB,IFF,HES,AEP
PCAR,0.069437,0.011799,0.007214,0.009742,0.018730,0.010066,0.008346,0.009538,0.020510,0.015664,0.016691,0.007782,0.008225,0.007181,0.005824
GIS,0.011799,0.053599,0.008817,0.014527,0.007301,0.008566,0.018381,0.010116,0.019771,0.024974,0.017946,0.008836,0.011655,0.005640,0.007983
BKR,0.007214,0.008817,0.118552,0.003650,0.001159,0.000768,0.003649,0.010350,0.004596,0.010473,0.009700,0.003309,0.006550,0.033311,0.004063
HRB,0.009742,0.014527,0.003650,0.061112,0.006877,0.004921,0.007370,0.011769,0.018448,0.017833,0.016909,0.001625,0.008710,0.003290,0.007499
CMI,0.018730,0.007301,0.001159,0.006877,0.081514,0.007798,0.009185,0.011297,0.016215,0.012050,0.020170,0.009766,0.009222,0.000881,0.003183
CINF,0.010066,0.008566,0.000768,0.004921,0.007798,0.042698,0.002482,0.007408,0.006494,0.006741,0.008289,0.005080,0.005688,0.001625,0.002936
LLY,0.008346,0.018381,0.003649,0.007370,0.009185,0.002482,0.062422,0.007692,0.016172,0.026538,0.011632,0.003810,0.007586,0.005255,0.003780
UTX,0.009538,0.010116,0.010350,0.011769,0.011297,0.007408,0.007692,0.056012,0.005048,0.018238,0.013962,0.006738,0.008417,0.005672,0.005587
CMCSA,0.020510,0.019771,0.004596,0.018448,0.016215,0.006494,0.016172,0.005048,0.202336,0.022510,0.028264,0.001610,0.011753,0.013260,0.005837
LB,0.015664,0.024974,0.010473,0.017833,0.012050,0.006741,0.026538,0.018238,0.022510,0.142771,0.023028,0.002669,0.012977,0.012005,0.008985


### Using An Optimizer
- Initialization
- Setting up an Objective
- Setting up Constraints
- Calculate an optimal portfolio allocation
- Return Summarized Results


#### Initialization
- Setting Up An Optimizer with Mean Return, Covariance Matrix and Beta
- Beta is optional, but if no beta is given then no constraints/objectives that use beta can be performed

In [53]:
# Without Beta
PortOpt = Optimizer(mu_return, cov_matrix)

In [54]:
# With Beta
PortOpt = Optimizer(mu_return, cov_matrix, beta_vec)

#### Objective Summary
- Optimization are performed with scipy.optimize method using 'SLSQP'
- In certain scenarios optimization may get stuck in the local minima but mostly optimization are successful
- All builtin Objectives can be checked by calling objective_options()
- Customizable Objectives also available

##### Risk Related (Concerned with Covariance Only)
   - Minimize Correlation Factor,
   - Maximize Diversification Factor,
   - Equal Risk Parity
   - Minimize Volatility
   
##### Risk-Reward Metrics (Concerned with Return and Risk)
   - Efficient Frontier (Most Classic Version of Mean-Variance Optimization)
   - Minimize Portfolio Beta (Market Neutral)
   - Maximize Sharpe Ratio
   - Maximize Treynor Ratio
   - Maximize Jenson's Alpha
   
##### Numerically Calculated (No Optimization Problem)
   - Assign Portfolio Weights Based on Inverse Volatility/Variance
   - Assign Portfolio Weights Equally/Based on Market Capitalization (Long Only)

In [55]:
PortOpt.objective_options()

{'efficient_frontier': <Signature (w, aversion)>,
 'equal_risk_parity': <Signature (w)>,
 'min_correlation': <Signature (w)>,
 'min_volatility': <Signature (w)>,
 'min_variance': <Signature (w)>,
 'min_skew': <Signature (w)>,
 'min_kurt': <Signature (w)>,
 'min_moment': <Signature (w)>,
 'max_return': <Signature (w)>,
 'max_diversification': <Signature (w)>,
 'max_sharpe': <Signature (w, risk_free)>,
 'min_beta': <Signature (w)>,
 'max_treynor': <Signature (w, risk_free)>,
 'max_jenson_alpha': <Signature (w, risk_free, market_return)>,
 'inverse_volatility': <Signature ()>,
 'inverse_variance': <Signature ()>,
 'equal_weight': <Signature (leverage)>,
 'market_cap_weight': <Signature (leverage)>}

#### Constraint Summary
- All builtin Constraints can be checked by calling constraint_options()
- Customizable Constraints also available
- Constraints are typically set with a tuple of (lower_bound, upper_bound) but if you pass in just one bound, the Optimizer will assume you passed in a lower/upper bound based on the metric

##### Risk Related (Concerned with Covariance Only)
   - Constraining Volatility 
   
##### Risk-Reward Metrics (Concerned with Return and Risk)
   - Constraining Portfolio Beta
   - Constraining Sharpe Ratio  (Default is a lower bound)
   - Constraining Treynor Ratio (Default is a lower bound)
   - Constraining Jenson's Alpha (Default is a lower bound)
    
##### Portfolio Composition
   - Individual Weight (Pass in a weight bound that applies to all assets or each individual can be capped/floored)
   - Total Leverage
   - Number of Assets to Hold
   - Concentration of Positions

In [56]:
PortOpt.constraint_options()

{'weight': <Signature (weight_bound, leverage)>,
 'num_assets': <Signature (num_assets)>,
 'concentration': <Signature (top_holdings, top_concentration)>,
 'expected_return': <Signature (bound)>,
 'sharpe': <Signature (risk_free, bound)>,
 'beta': <Signature (bound)>,
 'treynor': <Signature (bound, risk_free)>,
 'jenson_alpha': <Signature (bound, risk_free, market_return)>,
 'volatility': <Signature (bound)>,
 'variance': <Signature (bound)>,
 'skew': <Signature (bound)>,
 'kurt': <Signature (bound)>,
 'moment': <Signature (bound)>}

#### Example: Minimize Portfolio Volatility (Long-only, No Leverage)
- Set Weight Bound to be (0, 1) and leverage = 1
- Set Objective to min_volatility
- Pass in additional parameters to summary to construct more metrics

In [57]:
PortOpt.add_objective("min_volatility")
PortOpt.add_constraint("weight", weight_bound=(0,1), leverage=1)

In [58]:
PortOpt.solve() # Will Raise Error if not solvable

In [59]:
weight_dict, metric_dict = PortOpt.summary()

In [63]:
metric_dict

{'Expected Return': 0.0857,
 'Leverage': 1.0,
 'Number of Holdings': 12,
 'Volatility': 0.0951,
 'Portfolio Beta': 0.6613}

In [64]:
weight_dict

{'PCAR': 0.0158,
 'BKR': 0.0203,
 'HRB': 0.0438,
 'CMI': 0.0388,
 'CINF': 0.1306,
 'LLY': 0.0708,
 'UTX': 0.038,
 'MXIM': 0.0062,
 'USB': 0.0649,
 'IFF': 0.1352,
 'HES': 0.0909,
 'AEP': 0.3447}

In [65]:
weight_dict, metric_dict = PortOpt.summary(risk_free=0.02)

In [66]:
metric_dict

{'Expected Return': 0.0857,
 'Leverage': 1.0,
 'Number of Holdings': 12,
 'Volatility': 0.0951,
 'Portfolio Beta': 0.6613,
 'Sharpe Ratio': 0.6908,
 'Treynor Ratio': 0.0994}

In [67]:
weight_dict, metric_dict = PortOpt.summary(risk_free=0.02, market_return=0.07)

In [68]:
metric_dict

{'Expected Return': 0.0857,
 'Leverage': 1.0,
 'Number of Holdings': 12,
 'Volatility': 0.0951,
 'Portfolio Beta': 0.6613,
 'Sharpe Ratio': 0.6908,
 'Treynor Ratio': 0.0994,
 "Jenson's Alpha": 0.0327}

#### Example: Efficient Frontier (Short-selling Permitted)
- Set Weight Bound to be (-1, 1) and leverage = 1
- Set Objective to efficient_frontier, aversion=2.5/0.2 (more risk averse, less risk averse)
- Pass in additional parameters to summary to construct more metrics
- As you can see, when investor does not care about risk aversion (aversion=0.2), the optimized portfolio is holding very concentrated positions in one/two stocks

In [79]:
PortOpt.add_objective("efficient_frontier", aversion=2.5)
PortOpt.add_constraint("weight", weight_bound=(-1,1), leverage=1)
PortOpt.solve()
weight_dict, metric_dict = PortOpt.summary(risk_free=0.02, market_return=0.07)

In [80]:
weight_dict

{'HRB': 0.1768, 'CINF': 0.3581, 'CMCSA': 0.0422, 'MXIM': 0.4229}

In [81]:
metric_dict

{'Expected Return': 0.3448,
 'Leverage': 1.0,
 'Number of Holdings': 4,
 'Volatility': 0.214,
 'Portfolio Beta': 1.0853,
 'Sharpe Ratio': 1.5179,
 'Treynor Ratio': 0.2993,
 "Jenson's Alpha": 0.2705}

In [82]:
PortOpt.add_objective("efficient_frontier", aversion=0.2)
PortOpt.add_constraint("weight", weight_bound=(-1,1), leverage=1)
PortOpt.solve()
weight_dict, metric_dict = PortOpt.summary(risk_free=0.02, market_return=0.07)

In [83]:
weight_dict

{'MXIM': 1.0}

In [84]:
metric_dict

{'Expected Return': 0.5169,
 'Leverage': 1.0,
 'Number of Holdings': 1,
 'Volatility': 0.4156,
 'Portfolio Beta': 1.5582,
 'Sharpe Ratio': 1.1955,
 'Treynor Ratio': 0.3189,
 "Jenson's Alpha": 0.4189}

#### Example: Maximize Sharpe (Short-selling Permitted with Leverage and Capped Portfolio Weight)
- Set Weight Bound to be (-0.25, 0.25) and leverage = 2
- Set Objective to max_sharpe
- Pass in risk_free rate of return (Must be positive in accordance with CAPM assumptions)
- As you can see, this portfolio has a relatively high sharpe ratio

In [85]:
PortOpt.add_objective("max_sharpe", riskf_free=0.015)
PortOpt.add_constraint("weight", weight_bound=(-0.25,0.25), leverage=1)
PortOpt.solve()
weight_dict, metric_dict = PortOpt.summary(risk_free=0.015, market_return=0.07)

In [87]:
weight_dict

{'GIS': 0.0511,
 'BKR': -0.0266,
 'HRB': 0.1321,
 'CMI': 0.0454,
 'CINF': 0.2142,
 'LLY': -0.119,
 'UTX': -0.0622,
 'CMCSA': 0.0274,
 'LB': -0.0227,
 'MXIM': 0.1446,
 'USB': 0.0443,
 'IFF': 0.1105}

In [88]:
metric_dict

{'Expected Return': 0.1967,
 'Leverage': 1.0001,
 'Number of Holdings': 12,
 'Volatility': 0.1038,
 'Portfolio Beta': 0.4797,
 'Sharpe Ratio': 1.7509,
 'Treynor Ratio': 0.3788,
 "Jenson's Alpha": 0.1553}

#### Example: Maximize Treynor  (Capped Portfolio Weight, Maximum # of Holdings, Maximum Concentration)
<br>
*** Note that Maximization of Treynor can only happen in a long-only portfolio
<br>
*** The reason is because Treynor is calculated by dividing excess return over beta and with a short-selling permitted portfolio.
<br>
*** beta can infinitely approach 0 and Treynor will be infinitely large

- Set Weight Bound to be (0.0, 1) and leverage = 2
- Set a Maximum of 10 Holdings
- Set a Maximum Concentration of Top 2 Holdings Less than 50% of the entire portfolio
- Set Objective to max_treynor
- Pass in risk_free rate of return


In [106]:
PortOpt.add_objective("max_treynor", risk_free=0.015)
PortOpt.add_constraint("weight", weight_bound=(0, 1), leverage=1)
PortOpt.add_constraint("concentration", top_holdings=2, top_concentration=0.6)
PortOpt.add_constraint("num_assets", num_assets=5)
PortOpt.solve()
weight_dict, metric_dict = PortOpt.summary(risk_free=0.015, market_return=0.07, top_holdings=2)

In [107]:
metric_dict

{'Expected Return': 0.085,
 'Leverage': 1.0001,
 'Number of Holdings': 5,
 'Top 2 Holdings Concentrations': 0.5779,
 'Volatility': 0.1253,
 'Portfolio Beta': 0.7574,
 'Sharpe Ratio': 0.5586,
 'Treynor Ratio': 0.0924,
 "Jenson's Alpha": 0.0283}

In [108]:
weight_dict

{'GIS': 0.309, 'CINF': 0.0505, 'USB': 0.104, 'HES': 0.2676, 'AEP': 0.269}

#### Example: Minimize Beta  (Capped Portfolio Weight, Maximum # of Holdings, Maximum Concentration)

- Set Weight Bound to be (-1, 1) and leverage = 2
- Set a Maximum of 10 Holdings
- Set a Maximum Concentration of Top 2 Holdings Less than 50% of the entire portfolio
- Set Objective to min_beta
- As you can see, with short-selling permitted, the beta of the portfolio will be close to 0, will also horrible expected return, so you can add an additional constraint to set a lower bound for expected return and the result will show that portfolio achieves the minimum expected return while maintaining a low beta

In [109]:
PortOpt.add_objective("min_beta")
PortOpt.add_constraint("weight", weight_bound=(-1, 1), leverage=1)
PortOpt.add_constraint("concentration", top_holdings=2, top_concentration=0.5)
PortOpt.add_constraint("num_assets", num_assets=10)
PortOpt.solve()
weight_dict, metric_dict = PortOpt.summary(risk_free=0.015, market_return=0.07, top_holdings=2)

In [110]:
weight_dict

{'GIS': 0.0723,
 'CINF': -0.0413,
 'LLY': 0.0654,
 'CMCSA': -0.0976,
 'LB': 0.092,
 'MXIM': 0.1103,
 'USB': -0.0895,
 'IFF': -0.1267,
 'HES': -0.1425,
 'AEP': -0.1626}

In [111]:
metric_dict

{'Expected Return': -0.0009,
 'Leverage': 1.0002,
 'Number of Holdings': 10,
 'Top 2 Holdings Concentrations': 0.305,
 'Volatility': 0.0896,
 'Portfolio Beta': -0.0,
 'Sharpe Ratio': -0.1774,
 'Treynor Ratio': 554.2507,
 "Jenson's Alpha": -0.0159}

In [115]:
PortOpt.add_objective("min_beta")
PortOpt.add_constraint("weight", weight_bound=(-1, 1), leverage=1)
PortOpt.add_constraint("concentration", top_holdings=2, top_concentration=0.5)
PortOpt.add_constraint("num_assets", num_assets=10)
PortOpt.add_constraint("expected_return", bound=0.10)
PortOpt.solve()
weight_dict, metric_dict = PortOpt.summary(risk_free=0.015, market_return=0.07, top_holdings=2)

In [116]:
weight_dict

{'PCAR': 0.0456,
 'GIS': 0.081,
 'BKR': -0.1194,
 'CMI': 0.0908,
 'LLY': -0.1893,
 'UTX': -0.0801,
 'MXIM': 0.0647,
 'USB': 0.2072,
 'HES': -0.0628,
 'AEP': -0.0591}

In [117]:
metric_dict

{'Expected Return': 0.1005,
 'Leverage': 1.0,
 'Number of Holdings': 10,
 'Top 2 Holdings Concentrations': 0.3965,
 'Volatility': 0.1036,
 'Portfolio Beta': 0.0001,
 'Sharpe Ratio': 0.8251,
 'Treynor Ratio': 1355.7077,
 "Jenson's Alpha": 0.0854}

#### Example: Minimize Volatility  (with a Variety of Constraints)

- Set Weight Bound to be (0.0, 1) and leverage = 2
- Set a Maximum of 10 Holdings
- Set a Maximum Concentration of Top 2 Holdings Less than 50% of the entire portfolio
- Set a Minimum Expected Return Constraint
- Set a Minimum Sharpe Constraint
- Set a Maximum Beta Constraint
- Set Objective to min_volatility

##### Examine the Summary to see if the portfolio created aligns with your expectation
- Minimize Volatility Using 2X Leverage with Top 2 Holdings weighting less than 50% of the entire portfolio, With a Minimum Expected Return of 10% and Sharpe Ratio greater than 1

- As you can see, the Optimizer allows you to flexibly construct complex portfolio allocation problems given a variety types of constraints combined together


In [125]:
PortOpt.add_objective("min_volatility")
PortOpt.add_constraint("weight", weight_bound=(0, 1), leverage=2)
PortOpt.add_constraint("concentration", top_holdings=2, top_concentration=0.5)
PortOpt.add_constraint("expected_return", bound=0.10)
PortOpt.add_constraint("sharpe", bound=1, risk_free=0.015) #you can also set bound=(1, 2) to confine sharpe within a range
PortOpt.solve()
weight_dict, metric_dict = PortOpt.summary(risk_free=0.015, market_return=0.07, top_holdings=2)

In [126]:
weight_dict

{'PCAR': 0.0313,
 'BKR': 0.0298,
 'HRB': 0.1151,
 'CMI': 0.0841,
 'CINF': 0.2971,
 'LLY': 0.1035,
 'UTX': 0.0519,
 'CMCSA': 0.0057,
 'MXIM': 0.044,
 'USB': 0.1343,
 'IFF': 0.2849,
 'HES': 0.1751,
 'AEP': 0.6432}

In [127]:
metric_dict

{'Expected Return': 0.2065,
 'Leverage': 2.0,
 'Number of Holdings': 13,
 'Top 2 Holdings Concentrations': 0.4702,
 'Volatility': 0.1915,
 'Portfolio Beta': 1.3484,
 'Sharpe Ratio': 0.9999,
 'Treynor Ratio': 0.142,
 "Jenson's Alpha": 0.1173}

#### Example: Maximize Diversifcation  (with a Variety of Constraints)

- Set Weight Bound to be (0.0, 1) and leverage = 1.5
- Set a Maximum of 10 Holdings
- Set a Maximum Concentration of Top 5 Holdings Less than 70% of the entire portfolio
- Set a Minimum Expected Return Constraint
- Set a Minimum Treynor Constraint
- Set Objective to max_diversifcation

##### Examine the Summary to see if the portfolio created aligns with your expectation
- Minimize Volatility Using 1.5X Leverage with Top 5 Holdings weighting less than 70% of the entire portfolio, With a Minimum Expected Return of 10%

In [136]:
PortOpt.add_objective("max_diversification")
PortOpt.add_constraint("weight", weight_bound=(0, 1), leverage=1.5)
PortOpt.add_constraint("concentration", top_holdings=5, top_concentration=0.7)
PortOpt.add_constraint("num_assets", num_assets=10)
PortOpt.add_constraint("expected_return", bound=0.10)
PortOpt.solve()
weight_dict, metric_dict = PortOpt.summary(risk_free=0.015, market_return=0.07, top_holdings=5)

In [137]:
weight_dict

{'CMI': 0.1262,
 'LLY': 0.1342,
 'UTX': 0.1264,
 'CMCSA': 0.0912,
 'LB': 0.0657,
 'MXIM': 0.1008,
 'USB': 0.2044,
 'IFF': 0.1547,
 'HES': 0.2172,
 'AEP': 0.2792}

In [138]:
metric_dict

{'Expected Return': 0.1531,
 'Leverage': 1.5,
 'Number of Holdings': 10,
 'Top 5 Holdings Concentrations': 0.6598,
 'Volatility': 0.1724,
 'Portfolio Beta': 1.2928,
 'Sharpe Ratio': 0.8012,
 'Treynor Ratio': 0.1068,
 "Jenson's Alpha": 0.067}

##### Note that setting constraints for treynor/jenson's alpha may not work as expected due to differentiability issues in scipy.optimize method

##### Linear constraints such as expected return should work all the time

#### Numerically Calculated
- The main purpose of numerically calculated constraints is to serve as a benchmark for optimized problems

In [145]:
PortOpt.add_objective("inverse_volatility", leverage=1.5)
PortOpt.solve()
weight_dict, metric_dict = PortOpt.summary(risk_free=0.015, market_return=0.07, top_holdings=5)

C:\Projects\markowitz-optimization\markowitz\Optimizer.py:135: UserWarning:

The problem formulated is not an optimization problem and is calculated numerically



In [146]:
weight_dict

{'PCAR': 0.0962,
 'GIS': 0.1095,
 'BKR': 0.0736,
 'HRB': 0.1025,
 'CMI': 0.0888,
 'CINF': 0.1227,
 'LLY': 0.1015,
 'UTX': 0.1071,
 'CMCSA': 0.0564,
 'LB': 0.0671,
 'MXIM': 0.061,
 'USB': 0.08,
 'IFF': 0.1422,
 'HES': 0.1056,
 'AEP': 0.1859}

In [147]:
metric_dict

{'Expected Return': 0.1727,
 'Leverage': 1.5001,
 'Number of Holdings': 15,
 'Top 5 Holdings Concentrations': 0.4449,
 'Volatility': 0.1642,
 'Portfolio Beta': 1.3669,
 'Sharpe Ratio': 0.9603,
 'Treynor Ratio': 0.1154,
 "Jenson's Alpha": 0.0825}

In [148]:
### Equal Weight
PortOpt.add_objective("equal_weight", leverage=1.5)
PortOpt.solve()
weight_dict, metric_dict = PortOpt.summary(risk_free=0.015, market_return=0.07, top_holdings=5)

In [149]:
weight_dict

{'PCAR': 0.1,
 'GIS': 0.1,
 'BKR': 0.1,
 'HRB': 0.1,
 'CMI': 0.1,
 'CINF': 0.1,
 'LLY': 0.1,
 'UTX': 0.1,
 'CMCSA': 0.1,
 'LB': 0.1,
 'MXIM': 0.1,
 'USB': 0.1,
 'IFF': 0.1,
 'HES': 0.1,
 'AEP': 0.1}

In [150]:
metric_dict

{'Expected Return': 0.1931,
 'Leverage': 1.5,
 'Number of Holdings': 15,
 'Top 5 Holdings Concentrations': 0.3333,
 'Volatility': 0.1804,
 'Portfolio Beta': 1.4939,
 'Sharpe Ratio': 0.9873,
 'Treynor Ratio': 0.1192,
 "Jenson's Alpha": 0.0959}

### Coskewness & Cokurtosis
- Some objective functions may not work as expected for coskewness and cokurtosis (They should be mainly used to simulate scenarios)
- Most Risk-Reward Matrices Are not Available for Optimizing Coskewness/Cokurtosis
- Diversification/Correlation Works for Covariance Matrix Only

In [151]:
coskew_matrix = mom_generator.calc_coskew_mat(method='exp', decay=0.94, span=30)

In [152]:
PortOpt = Optimizer(mu_return, coskew_matrix, beta_vec)

In [154]:
PortOpt.add_objective("equal_weight", leverage=1.5)
PortOpt.solve()
weight_dict, metric_dict = PortOpt.summary()

In [155]:
metric_dict

{'Expected Return': 0.1931,
 'Leverage': 1.5,
 'Number of Holdings': 15,
 'Skewness': 0.0227,
 'Portfolio Beta': 1.4939}

In [163]:
PortOpt.add_objective("min_skew")
PortOpt.add_constraint("expected_return", bound=0.1)
PortOpt.add_constraint("concentration", top_holdings=5, top_concentration=0.7)
PortOpt.solve()
weight_dict, metric_dict = PortOpt.summary()

In [164]:
metric_dict

{'Expected Return': 0.1059,
 'Leverage': 1.0002,
 'Number of Holdings': 11,
 'Skewness': -0.0287,
 'Portfolio Beta': 0.8805}

In [165]:
weight_dict

{'PCAR': 0.067,
 'BKR': 0.067,
 'CMI': 0.0145,
 'LLY': 0.067,
 'CMCSA': 0.1377,
 'LB': 0.0176,
 'MXIM': 0.067,
 'USB': 0.067,
 'IFF': 0.067,
 'HES': 0.067,
 'AEP': 0.3614}

#### Minimize Portfolio Kurtosis
- Use a smaller timeframe because of computation expensiveness

In [12]:
mom_generator = MomentGenerator(daily_return.iloc[:100]) # 1000 Observations are too large for computation
cokurt_matrix = mom_generator.calc_cokurt_mat(method='exp', decay=0.94, span=30)

In [14]:
PortOpt = Optimizer(mu_return, cokurt_matrix)

C:\Projects\markowitz-optimization\markowitz\Optimizer.py:311: UserWarning:

"Detected no beta input. Will not be able to perform any beta-related optimization.



In [18]:
PortOpt.add_objective("min_kurt")
PortOpt.add_constraint("expected_return", bound=0.1)
PortOpt.add_constraint("concentration", top_holdings=5, top_concentration=0.7)
PortOpt.solve()
weight_dict, metric_dict = PortOpt.summary(top_holdings=5)

In [19]:
weight_dict

{'APA': 0.0939,
 'AON': 0.0109,
 'HUM': 0.0986,
 'ROL': 0.1403,
 'FISV': 0.0281,
 'PVH': 0.1406,
 'AJG': 0.1604,
 'VAR': 0.0998,
 'SNA': 0.112,
 'IBM': 0.0123,
 'DOV': 0.103}

In [20]:
metric_dict

{'Expected Return': 0.1704,
 'Leverage': 0.9999,
 'Number of Holdings': 11,
 'Top 5 Holdings Concentrations': 0.6564,
 'Kurtosis': 0.0819}

### Customized Objectives & Constraints

- The Optimizer class allows for Customized Objectives & Constraints with a few limitations
- Must be constructed in a specific format
    - custom_func(w, **kwargs)
- Cannot Use Attributes such as Covariance Matrix/Beta/Moment, Instead User must pass in everything in the customized function for either objectives/constraints

- Below is an example of a customized objective (may not convey any real meaning in finance)

In [27]:
PortOpt = Optimizer(mu_return, cov_matrix, beta_vec)

In [45]:
custom_const = [{'type': 'ineq', 'fun': lambda w: np.sum(w ** 2) - 0.25}]

# Maximizing
def custom_obj(w, risk_free):
    return -(w @ (mu_return ** 2) - risk_free * 5 - w @ cov_matrix @ w.T - w @ beta_vec)

In [46]:
PortOpt.add_objective("custom", custom_func=custom_obj, risk_free=0.05)
PortOpt.add_constraint("custom", custom_const=custom_const)

In [47]:
PortOpt.solve()

In [48]:
weight_dict, metric_dict = PortOpt.summary(risk_free=0.05, market_return=0.10, top_holdings=5)

In [49]:
weight_dict

{'FISV': 0.1949, 'AJG': 0.8051}

In [50]:
metric_dict

{'Expected Return': 0.1531,
 'Leverage': 1.0,
 'Number of Holdings': 2,
 'Top 5 Holdings Concentrations': 1.0,
 'Volatility': 0.2046,
 'Portfolio Beta': 0.4869,
 'Sharpe Ratio': 0.5039,
 'Treynor Ratio': 0.2118,
 "Jenson's Alpha": 0.0788}